In [1]:
  from google.colab import drive
drive.mount('/content/drive/')
import numpy as np
import itertools
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import pickle
import tensorflow as tf
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input 
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import regularizers
from tensorflow.python.keras.models import load_model
plt.style.use('seaborn-white')

MessageError: ignored

In [ ]:
def split_wav(data, sample_rate, start, end):
  start *= sample_rate
  end *= sample_rate
  return data[start:end]

In [ ]:
# "belt": 0, "straight": 1, "vibrato": 2, "vocal_fry": 3
sec = [[5,6], [8,9], [13, 14], [16,17], [17,18], [19,20], [25, 26], [27, 28], [28, 29], [30, 31], [32,33], [36,37], [38,39], [47,48], [49,50], [52,53], [58,59],
       [74,75], [79,80], [80,81], [85,86], [89,90], [93, 94],[95,95.5],[95.5,96], [96,97], [98,99], [99,100], [102,103], [104,105], [107,108], [113,114], [115,116],[118,119], [124,125],
       [141,142], [146,147],[148,149],[153,154],[160,160.5],[162,163],[170,171],[173,174]]
label = ["vocal_fry","vocal_fry","vocal_fry","vocal_fry","vocal_fry","vocal_fry","vibrato","belt",'belt','belt','vibrato','vibrato', 'belt','vibrato','belt','belt','vibrato',
         "vocal_fry","vocal_fry","vibrato","vocal_fry","vocal_fry",'belt','belt',"vocal_fry",'belt','vibrato', "vocal_fry",'belt','belt','belt','vibrato','belt','belt','vibrato',
         "vocal_fry", 'vibrato','vibrato','vibrato',
         'belt','belt','belt','belt']

In [ ]:
fine, sr = librosa.load("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/???.wav", sr=44100)
fine_features = []
for i in range(len(sec)):
  if (sec[i][1] - sec[i][0]) < 1:
    tmp = [0 * 44100]
    if type(sec[i][0]) != int:
      tmp[0:int(44100*0.5)] = [0 for _ in range(int(44100*0.5))]
      tmp[int(44100*0.5) : 44100] = fine[int(sec[i][0]*44100) : int(sec[i][1]*44100)]
    else:
      tmp[0:int(44100*0.5)] = fine[int(sec[i][0]*44100) : int(sec[i][1]*44100)]
      tmp[int(44100*0.5) : 44100] = [0 for _ in range(int(44100*0.5))]
    continue
  tmp = fine[int(sec[i][0]*44100) : int(sec[i][1]*44100)]
  fine_features.append([tmp, label[i]]) 

fine_df = pd.DataFrame(fine_features, columns = ['audio', 'label'])
fine_df.to_pickle("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/yj_fine_straight.pkl")

In [ ]:
fine_df = pd.read_pickle("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/yj_fine_straight.pkl")
audio = np.array(fine_df.audio.tolist())
label = np.array(fine_df.label.tolist())

In [ ]:
from sklearn.preprocessing import StandardScaler
def augmentation(audio, label):
  with tf.device('/GPU:0'):
    audio_pitch_shift = []
    for i in tqdm(range(len(audio))):
      audio_pitch_shift.append(librosa.effects.pitch_shift(audio[i], 44100, 4))

    noise = np.random.randn(len(audio[0]))
    noise = (noise - min(noise)) / (max(noise) - min(noise))
    audio_noise = audio + 0.1 * noise
    # Cast back to same data type
    audio_noise = audio_noise.astype(type(audio[0]))


    audio = np.concatenate([audio, audio_pitch_shift, audio_noise],  axis=0) 
    label = np.concatenate([label, label, label]) 

    scaler = StandardScaler()
    audio = scaler.fit_transform(audio)

    tmp = []
    for i in tqdm(range(len(audio))):
      tmp.append([audio[i], label[i]])
    tmpdf = pd.DataFrame(tmp, columns = ['audio', 'label'])
    tmpdf.to_pickle(f"/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/augmented_scaled_yj_fine_straight.pkl")

    return (audio, label)

augmentation(audio, label)

100%|██████████| 120/120 [00:00<00:00, 330694.14it/s]


(array([[-0.18226672, -0.31172336, -0.25614272, ..., -1.20221142,
         -1.38084045, -1.47723582],
        [-0.62514398, -0.72743355, -0.72137005, ..., -0.03330806,
         -0.13008191,  0.01951075],
        [-0.76769567, -0.86346032, -0.87560732, ...,  0.73412895,
          0.58470655,  0.74451979],
        ...,
        [ 0.02782124,  0.30070918,  0.22077653, ...,  0.06435202,
          0.36721135,  0.1981584 ],
        [ 0.33092424,  0.57183225,  0.4036174 , ..., -1.24348762,
         -1.0975112 , -1.41207015],
        [-2.83000044, -2.20452018, -2.4380253 , ...,  1.61278879,
          1.71867053,  1.56660069]]),
 array(['straight', 'straight', 'straight', 'straight', 'straight',
        'straight', 'straight', 'straight', 'straight', 'straight',
        'straight', 'straight', 'straight', 'straight', 'straight',
        'straight', 'straight', 'straight', 'straight', 'straight',
        'straight', 'straight', 'straight', 'straight', 'straight',
        'straight', 'straight', '

# MR 제거 테스트

In [ ]:
from IPython.display import Audio

In [ ]:
fine, sr = librosa.load("/content/drive/MyDrive/Startup_Hackathon_Jarvis/fine_data/fine.wav", sr=44100)

In [ ]:
S_full, phase = librosa.magphase(librosa.stft(fine))

In [ ]:
Audio(data=fine[10*sr:15*sr], rate=sr)

In [ ]:
S_filter = librosa.decompose.nn_filter(S_full,
                                       aggregate=np.median,
                                       metric='cosine',
                                       width=int(librosa.time_to_frames(2, sr=sr)))
S_filter = np.minimum(S_full, S_filter)

In [ ]:
margin_i, margin_v = 2, 6
power = 2

mask_i = librosa.util.softmask(S_filter,
                               margin_i * (S_full - S_filter),
                               power=power)

mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)

# Once we have the masks, simply multiply them with the input spectrum
# to separate the components

S_foreground = mask_v * S_full
S_background = mask_i * S_full

In [ ]:
y_foreground = librosa.istft(S_foreground * phase)
Audio(data=y_foreground[10*sr:15*sr], rate=sr)

In [ ]:
y_background = librosa.istft(S_background  * phase)
Audio(data=y_background[10*sr:15*sr], rate=sr)

In [ ]:
Audio(data=tmp, rate=44100)

In [ ]:
# https://source-separation.github.io/tutorial/first_steps/repetition.html 
# 대박 도움됨
!pip install nussl

In [ ]:
import nussl
fine_sig = nussl.AudioSignal(audio_data_array = fine, sample_rate=44100)
repet = nussl.separation.primitive.Repet(fine_sig)
bg_mask, fg_mask = repet.run()

In [ ]:
# Get the mask numpy arrays
bg_mask_arr = bg_mask.mask
fg_mask_arr = fg_mask.mask

In [ ]:
def apply_mask_with_noisy_phase(mix_stft, mask):
    mix_magnitude, mix_phase = np.abs(mix_stft), np.angle(mix_stft)
    src_magnitude = mix_magnitude * mask
    src_stft = src_magnitude * np.exp(1j * mix_phase)
    return src_stft

bg_stft = apply_mask_with_noisy_phase(fine_sig.stft_data, bg_mask_arr)
fg_stft = apply_mask_with_noisy_phase(fine_sig.stft_data, fg_mask_arr)

TypeError: ignored

In [ ]:
print('REPET Foreground with Phase')
_ = fg_phase.embed_audio()

In [ ]:
!pip install viz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for viz: filename=viz-0.0.5-py3-none-any.whl size=9090 sha256=8984a36d8fff6e628977fa32fe932d38eb50fa0edb1c3ed223225e036becc67f
  Stored in directory: /root/.cache/pip/wheels/8a/f4/1c/ec6953877fdb277446ff6950bf35c8f4a2b7ce7829982300ac
Successfully built viz


In [ ]:
import nussl
import viz
fine_sig = nussl.AudioSignal(audio_data_array = fine, sample_rate=44100)
repet = nussl.separation.primitive.Repet(fine_sig)
repet_bg, repet_fg = repet()
viz.show_sources({'Background': repet_bg, 'Foreground': repet_fg})

AttributeError: ignored

In [ ]:
Audio(data=repet_fg, rate=44100)

TypeError: ignored